# Embedding Model 변경 (intfloat/multilingual-e5-large)

In [ ]:
# ✅ 1️⃣ 데이터 로드 (박물관 데이터)
data_path = "merged_museum_data_rag.csv"  # ✅ 본인 데이터 경로 확인!
df = pd.read_csv(data_path)

# ✅ 문장 분할 함수 (간단한 마침표 기준 + 공백 제거)
def split_sentences(text):
    # 마침표, 느낌표, 물음표 기준으로 문장 분리 (뒤에 공백도 포함)
    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    return [s for s in sentences if s]  # 빈 문자열 제거

# ✅ 2️⃣ Title + Description을 문장 단위로 쪼개서 문서 생성
documents = []
for _, row in df.iterrows():
    title = row["Title"]
    description = row["Description"]
    sentences = split_sentences(description)

    for sentence in sentences:
        documents.append({
            "text": f"{title} - {sentence}",
            "metadata": {"title": title}
        })

# ✅ 3️⃣ 임베딩 모델
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

# ✅ 4️⃣ FAISS 벡터스토어 생성
vectorstore = FAISS.from_texts(
    [doc["text"] for doc in documents],
    embedding_model,
    metadatas=[doc["metadata"] for doc in documents]
)

# ✅ 5️⃣ 저장
vectorstore.save_local("faiss_index")
print("✅ 문장 단위 벡터DB 생성 완료!")